In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re

## Intro to NLP

So far in this course we’ve worked with information that has been at least minimally processed: the real world translated into numerical values and organized into variables.  But the majority of the information available in the world isn’t numeric – it’s verbal.  From snarky movie reviews:

>_"Deuce Bigalow: European Gigolo" makes a living cleaning fish tanks and occasionally prostituting himself. How much he charges I'm not sure, but the price is worth it if it keeps him off the streets and out of another movie. "Deuce Bigalow" is aggressively bad, as if it wants to cause suffering to the audience. The best thing about it is that it runs for only 75 minutes. [...] Speaking in my official capacity as a Pulitzer Prize winner, Mr. Schneider, your movie sucks._ – [Roger Ebert](http://www.rogerebert.com/reviews/deuce-bigalow-european-gigolo-2005).

and controversial tweets:

>_Happy New Year to all, including to my many enemies and those who have fought me and lost so badly they just don't know what to do. Love!”_ - [Donald Trump](https://twitter.com/realDonaldTrump/status/815185071317676033)

to speeches and the dialogue on tv shows and movies:

>_In this world there is room for everyone, and the good earth is rich and can provide for everyone. The way of life can be free and beautiful, but we have lost the way... We have developed speed, but we have shut ourselves in. Machinery that gives abundance has left us in want. Our knowledge has made us cynical; our cleverness, hard and unkind. We think too much and feel too little. More than machinery, we need humanity. More than cleverness, we need kindness and gentleness._ – [Charlie Chaplin](https://www.youtube.com/watch?v=J7GY1Xg6X20)

The world is full of words and those words are full of data.

## Processing and analysis

NLP is a two-part problem: First, process the data from its original form (blocks of text or speech) into a form the computer can understand, then conduct analysis on the processed data. Step one, which we will address in this assignment, may involve elements of data cleaning and feature extraction. When dealing with verbal information, this step is called _language parsing_.  Domain knowledge (information about word frequency, meaning, and grammar) is applied to raw text to extract features of interest.

You may recall that in Unit 2, we built a spam filter with a Naive Bayes classifier. The spam filter parsed individual email messages and created a series of binary features indicating the presence or absence of a set of keywords that we assumed would discriminate between spam and non-spam emails. The process of coding the presence or absence of keywords is a very simple example of language parsing. Now, we are going to go much deeper.

We'll work with two different NLP packages: [NLTK](http://www.nltk.org/) and [spaCy](https://spacy.io).  NLTK, or Natural Language ToolKit, is a seasoned package with great richness and depth. It is good for learning language parsing because it is highly customizeable and transparent. On the other hand, it also contains many older models and methods that are useful for teaching NLP but are not optimal for production code. spaCy is almost the direct opposite. Rather than offering language parsing options, spaCy just processes text data using whatever algorithms and methods are considered "state of the art". It is considerably leaner, and because it is written in Cython (meaning Python code is translated into C and then run), it is considerably faster. On the other hand, we lose the virtue of choice, and if spaCy's algorithms change, our results could change as well.

In this lesson we'll use some of the text corpora (bodies of text) from NLTK, but will process them with spaCy. We'll also use regular expressions (the standard library package `re`) when we want to pull a very specific element of text out of a string, usually before passing the text to spaCy.

### Setup

Let's begin. First, if you haven't used NLTK yet, you'll want to install the package with `pip install nltk`. Then, run the cell below to launch an [interactive installer](http://www.nltk.org/data.html#interactive-installer). Using the installer, choose the "corpora" tab and download the "gutenberg" and "stopwords" corpora.

Note: Be sure that if you're using a mouse with a wheel that you drag the sidebar of the corpora screen by dragging and dropping it, not using the wheel.  [Some users have reported](https://github.com/nltk/nltk/issues/1673) that using the mouse wheel causes a UnicodeDecodeError.

In [2]:
import nltk
# Launch the installer to download "gutenberg" and "stop words" corpora.
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

### Cleaning
Now that we've acquired some data, let's dig in to look at it and get ready to clean things up. We're going to work specifically with two novels from the project Gutenberg corpora: _Alice in Wonderland_ by Lewis Carroll, and _Persuasion_ by Jane Austin. 

In [3]:
# Import the data we just downloaded and installed.
from nltk.corpus import gutenberg, stopwords

# Grab and process the raw data.
print(gutenberg.fileids())

persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# Print the first 100 characters of Alice in Wonderland.
print('\nRaw:\n', alice[0:100])

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']

Raw:
 [Alice's Adventures in Wonderland by Lewis Carroll 1865]

CHAPTER I. Down the Rabbit-Hole

Alice was


We're going to use _regular expressions_ (specifically [re.sub()](https://docs.python.org/3/library/re.html#re.sub), short for "substitute") to identify and remove substrings we don't want. Specifically we'll match those substrings with a regular expression and substitute in an empty string for them.

We won't go into detail here about how regular expressions work, but you should be able to get a good sense for what's happening by reading the code. If you want more information the [Python Regular Expression HOWTO](https://docs.python.org/3/howto/regex.html) is an accessible starting point and reference, and [RegExr](http://regexr.com/) is a useful tool for visualizing and tinkering with regular expressions.

We'll start our cleaning by removing the title. We'll match all text between square brackets and replace it with an empty string.

In [4]:
# This pattern matches all text between square brackets.
pattern = "[\[].*?[\]]"
persuasion = re.sub(pattern, "", persuasion)
alice = re.sub(pattern, "", alice)

# Print the first 100 characters of Alice again.
print('Title removed:\n', alice[0:100])

Title removed:
 

CHAPTER I. Down the Rabbit-Hole

Alice was beginning to get very tired of sitting by her sister on


In [5]:
# Now we'll match and remove chapter headings.
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)

# Ok, what's it look like now?
print('Chapter headings removed:\n', alice[0:100])

Chapter headings removed:
 



Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothin


In [6]:
# Remove newlines and other extra whitespace by splitting and rejoining.
persuasion = ' '.join(persuasion.split())
alice = ' '.join(alice.split())

# All done with cleanup? Let's see how it looks.
print('Extra whitespace removed:\n', alice[0:100])

Extra whitespace removed:
 Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to


Alrighty, that text is looking pretty good now.

## What information can we extract from text?

### Tokens

Each individual meaningful piece from a text is called a _token_, and the process of breaking up the text into these pieces is called _tokenization_. Tokens are generally words and punctuation. We may discard some tokens, such as punctuation, that we don't think add informational value. One class of potentially-uninformative tokens is _stop words_, words used very frequently that don't have much informational value, such as "the" and "of". Some NLP approaches discard stop words, while other approaches retain them because stop words can make up part of meaningful phrases ("master of the universe" being more specific and informative than "master" and "universe" alone).

In [7]:
# Here is a list of the stopwords identified by NLTK.
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Let's go ahead and use spaCy to parse our novels into tokens. When we call spaCy on the novel it will immediately and automatically parse it, tokenizing the string by breaking it into words and punctuation (and many other things we will explore).

Now is a good time to run `pip install spacy` in your terminal if you don't have it yet, then follow that up with `python -m spacy download 'en'` to download the individual spaCy English module.

In [8]:
import spacy
nlp = spacy.load('en')

# All the processing work is done here, so it may take a while.
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [9]:
# Let's explore the objects we've built.
print("The alice_doc object is a {} object.".format(type(alice_doc)))
print("It is {} tokens long".format(len(alice_doc)))
print("The first three tokens are '{}'".format(alice_doc[:3]))
print("The type of each token is {}".format(type(alice_doc[0])))

The alice_doc object is a <class 'spacy.tokens.doc.Doc'> object.
It is 34430 tokens long
The first three tokens are 'Alice was beginning'
The type of each token is <class 'spacy.tokens.token.Token'>


We see from introspecting the spaCy objects above that we're playing around with [doc](https://spacy.io/docs/api/doc) and [token](https://spacy.io/docs/api/token) objects. That's nice, but what can we _do_ with them?

A simple way to extract information from tokenized text data is to just count how often various tokens occur in each piece of text.

In [10]:
from collections import Counter

# Utility function to calculate how frequently words appear in the text.
def word_frequencies(text, include_stop=True):
    
    # Build a list of words.
    # Strip out punctuation and, optionally, stop words.
    words = []
    for token in text:
        if not token.is_punct and (not token.is_stop or include_stop):
            words.append(token.text)
            
    # Build and return a Counter object containing word counts.
    return Counter(words)
    
# The most frequent words:
alice_freq = word_frequencies(alice_doc).most_common(10)
persuasion_freq = word_frequencies(persuasion_doc).most_common(10)
print('Alice:', alice_freq)
print('Persuasion:', persuasion_freq)

Alice: [('the', 1524), ('and', 796), ('to', 724), ('a', 611), ('I', 534), ('it', 524), ('she', 508), ('of', 499), ('said', 453), ('Alice', 394)]
Persuasion: [('the', 3120), ('to', 2775), ('and', 2738), ('of', 2563), ('a', 1529), ('in', 1346), ('was', 1329), ('had', 1177), ('her', 1159), ('I', 1121)]


Those word counts aren't very informative. Most of them are stop words. Let's try again, leaving those out.

In [11]:
# Use our optional keyword argument to remove stop words.
alice_freq = word_frequencies(alice_doc, include_stop=False).most_common(10)
persuasion_freq = word_frequencies(persuasion_doc, include_stop=False).most_common(10)
print('Alice:', alice_freq)
print('Persuasion:', persuasion_freq)

Alice: [('I', 534), ('said', 453), ('Alice', 394), ("n't", 215), ("'s", 190), ('little', 124), ('The', 102), ('like', 84), ('went', 83), ('know', 83)]
Persuasion: [('I', 1121), ('Anne', 497), ("'s", 485), ('She', 326), ('Captain', 297), ('Mrs', 291), ('Elliot', 288), ('Mr', 255), ('He', 225), ('Wentworth', 217)]


That's better. Now let's identify which words are more characteristic of one text than another. Specifically, let's remove the words that are in the top ten for both books.

In [12]:
# Pull out just the text from our frequency lists.
alice_common = [pair[0] for pair in alice_freq]
persuasion_common = [pair[0] for pair in persuasion_freq]

# Use sets to find the unique values in each top ten.
print('Unique to Alice:', set(alice_common) - set(persuasion_common))
print('Unique to Persuasion:', set(persuasion_common) - set(alice_common))

Unique to Alice: {'said', 'know', 'The', 'little', "n't", 'went', 'Alice', 'like'}
Unique to Persuasion: {'She', 'He', 'Mr', 'Wentworth', 'Anne', 'Captain', 'Elliot', 'Mrs'}


And there you go. At this point we've got a pretty good collection of common, characteristic words for each novel.

## Lemmas

So far we've just looked at whether certain words are present and how frequently they appear. We can process these words further to remove a little more noise from our data. Consider the words "think", "thought", and "thinking". They're related. They all share the same root word: the verb "think". Sometimes we want to focus on the fact that the act of thinking comes up a lot in data, and not have that information split across all the different forms of "think".

To focus in like this, we can reduce each word to its root, or [_lemma_](https://simple.wikipedia.org/wiki/Lemma_%28linguistics%29), and do our counts again. This time we're building a count of _concepts_ rather than just _words_:

In [13]:
# Utility function to calculate how frequently lemas appear in the text.
def lemma_frequencies(text, include_stop=True):
    
    # Build a list of lemas.
    # Strip out punctuation and, optionally, stop words.
    lemmas = []
    for token in text:
        if not token.is_punct and (not token.is_stop or include_stop):
            lemmas.append(token.lemma_)
            
    # Build and return a Counter object containing word counts.
    return Counter(lemmas)

# Instantiate our list of most common lemmas.
alice_lemma_freq = lemma_frequencies(alice_doc, include_stop=False).most_common(10)
persuasion_lemma_freq = lemma_frequencies(persuasion_doc, include_stop=False).most_common(10)
print('\nAlice:', alice_lemma_freq)
print('Persuasion:', persuasion_lemma_freq)

# Again, identify the lemmas common to one text but not the other.
alice_lemma_common = [pair[0] for pair in alice_lemma_freq]
persuasion_lemma_common = [pair[0] for pair in persuasion_lemma_freq]
print('Unique to Alice:', set(alice_lemma_common) - set(persuasion_lemma_common))
print('Unique to Persuasion:', set(persuasion_lemma_common) - set(alice_lemma_common))


Alice: [('-PRON-', 758), ('say', 476), ('alice', 396), ('be', 254), ('not', 231), ('go', 133), ('think', 131), ('little', 126), ('the', 109), ('look', 105)]
Persuasion: [('-PRON-', 2241), ('anne', 497), ("'s", 466), ('captain', 303), ('elliot', 295), ('mrs', 291), ('good', 289), ('know', 258), ('think', 256), ('mr', 255)]
Unique to Alice: {'alice', 'go', 'be', 'little', 'the', 'look', 'say', 'not'}
Unique to Persuasion: {'know', 'good', "'s", 'mrs', 'elliot', 'anne', 'captain', 'mr'}


In addition to looking at lemmas, we could perform a similar analysis and pull out prefixes (`token.prefix_`) or suffixes (`token.suffix_`).

## Sentences

Beyond individual words, text can also be considered at the level of sentences. Using punctuation cues, we can split up text into sentences. Each sentence can then be summarized by, for example, using sentiment analysis to categorize sentences as having positive or negative sentiment. We may also be interested in how long sentences tend to be, and how many unique words make up a sentence.  The sentence also provides _context_ for the individual words, allowing us to draw even more information from each word.

We get a lot of automatic sentence-level information from spaCy. The `doc.sents` property will give us each sentence as a [span](https://spacy.io/docs/api/span) object. Let's look at some of that.

In [14]:
# Initial exploration of sentences.
sentences = list(alice_doc.sents)
print("Alice in Wonderland has {} sentences.".format(len(sentences)))

example_sentence = sentences[2]
print("Here is an example: \n{}\n".format(example_sentence))

Alice in Wonderland has 1678 sentences.
Here is an example: 
There was nothing so VERY remarkable in that; nor did Alice think it so VERY much out of the way to hear the Rabbit say to itself, 'Oh dear!



In [15]:
# Look at some metrics around this sentence.
example_words = [token for token in example_sentence if not token.is_punct]
unique_words = set([token.text for token in example_words])

print(("There are {} words in this sentence, and {} of them are"
       " unique.").format(len(example_words), len(unique_words)))

There are 29 words in this sentence, and 25 of them are unique.


## Parts of speech, dependencies, entities
Tokens within each sentence are also coded with the parts of speech they play. This is useful for distinguishing between _homographs_, words with the same spelling but different meaning (the umbrella term for this kind of linguistic feature is _polysemy_).  For example, the word "break" is a noun in "I need a break" but a verb in "I need to break the glass".

In [16]:
print(nlp("I need a break")[3].pos_)
print(nlp("I need to break the glass")[3].pos_)

NOUN
VERB


In [17]:
# View the part of speech for some tokens in our sentence.
print('\nParts of speech:')
for token in example_sentence[:9]:
    print(token.orth_, token.pos_)


Parts of speech:
There ADV
was VERB
nothing NOUN
so ADV
VERY ADV
remarkable ADJ
in ADP
that DET
; PUNCT


You also get _dependencies_, or how words relate to each other syntatically, with spaCy. Dependencies are a bit complicated – for a visual example of dependencies expressed as a tree, check the [About section of the Standford NLP Group Dependencies page](https://nlp.stanford.edu/software/stanford-dependencies.shtml). Stanford's NLP Group has had a lot of influence in this field, so you're likely to run across them frequently if you go deep into NLP. We aren't going to cover this in depth here.

Let's look at some dependencies of this sentence.

In [18]:
# View the dependencies for some tokens.
print('\nDependencies:')
for token in example_sentence[:9]:
    print(token.orth_, token.dep_, token.head.orth_)


Dependencies:
There expl was
was ROOT was
nothing attr was
so advmod remarkable
VERY advmod remarkable
remarkable amod nothing
in prep nothing
that pobj in
; punct was


Finally, spaCy gives us access to the named entities with `.ents`. In the example below you'll see some errors creep in – we can see that the entity identification rules in spaCy assume that, if it doesn't fall under any other obvious rule, any word or phrase IN ALL CAPS is an organization (if a noun) or an event (if a verb).

In [19]:
# Extract the first ten entities.
entities = list(alice_doc.ents)[0:10]
for entity in entities:
    print(entity.label_, ' '.join(t.orth_ for t in entity))

PERSON Alice
DATE the hot day
PERSON Alice
PRODUCT Rabbit
PRODUCT Rabbit
PRODUCT WAISTCOAT - POCKET
PERSON Alice
PERSON Alice
PERSON Alice
ORDINAL First


In [20]:
# All of the uniqe entities spaCy thinks are people.
people = [entity.text for entity in list(alice_doc.ents) if entity.label_ == "PERSON"]
print(set(people))

{'Serpent', 'The White Rabbit', 'Seaography', 'the Lobster Quadrille', 'Latitude', 'Bill', 'Beau', 'Canary', 'Curiouser', 'Mabel', "the King: '", 'Stand', 'M--', 'began:--', 'Gryphon', 'Lacie', 'Treacle', 'Longitude', 'the King', "the Duchess: '", 'Drink', 'Idiot', 'William the Conqueror', 'Shall', 'Morcar', 'Stolen', 'HAD', 'FUL SOUP', 'Pat', 'Down', 'Fury', 'Rabbit', 'Beautiful Soup', 'Sentence', 'WILLIAM', 'Edwin', 'YOURS', 'Ma', 'Pinch', 'Turtle Soup', '--or', 'Shakespeare', 'the March Hare', 'Footman', 'Sha', 'William', "the Mock Turtle: '", 'Fetch', 'the Lobster Quadrille?', 'Panther', 'The Queen', 'Duck', 'Frog-Footman', 'the Duchess', 'The Mock Turtle', 'the Queen of Hearts', 'Duchess', 'Stupid', "Don't", 'Run', 'Jack', 'INSIDE', 'Soles', 'indeed:--', 'this:--', 'Edgar Atheling', 'Soup of the evening', 'Begin', 'Fish-Footman', 'Prizes', 'Queen', 'Tillie', 'Soo', 'Alice', 'Sixteenth', 'Ou', 'Repeat', 'Cheshire Puss', 'Elsie', 'Hjckrrh', 'the White Rabbit', 'm--', 'the Mock Turtl

## The Pen is Mightier

Hopefully at this point it's clear that we have the ability to programmatically extract _a lot_ of information from text. Like any other feature extraction problem, let ingenuity be your guide. In the next assignment, we'll use this information to build a few supervised learning models.